# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada.csv', sep = '|')
df.head()

,acordao,arquivo,texto,areas
0,297/2016-P,547240.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...,Responsabilidade
1,366/2016-P,549518.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...,Finanças Públicas
2,944/2016-P,554399.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...,Responsabilidade
3,30/2016-P,545010.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...,Direito Processual
4,55/2016-P,544046.txt,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...,Pessoal


In [2]:
df.shape

(10524, 4)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [4]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 350000
limite_texto = 40000
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 344296 unique tokens.


In [5]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 40000)


In [6]:
x.shape, y.shape

((10524, 40000), (10524, 91))

# Treinamento

In [7]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D, Flatten
from keras.optimizers import RMSprop
from keras.layers.core import Dropout

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0309 19:48:10.639529 139653295728448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0309 19:48:10.886362 139653295728448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0309 19:48:10.934624 139653295728448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0309 19:48:10.990812 139653295728448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. P

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39994, 32)         22432     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7998, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 7992, 32)          7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 91)                3003      
Total params: 34,462,335
Trainable params: 34,462,335
Non-trainable params: 0
________________________________________________________________

W0309 19:48:12.242850 139653295728448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 61s 7ms/step - loss: 2.2959 - categorical_accuracy: 0.3570 - val_loss: 1.8042 - val_categorical_accuracy: 0.4708
Epoch 2/20
8419/8419 [==============================] - 44s 5ms/step - loss: 1.7796 - categorical_accuracy: 0.5181 - val_loss: 1.7499 - val_categorical_accuracy: 0.5064
Epoch 3/20
8419/8419 [==============================] - 44s 5ms/step - loss: 1.5874 - categorical_accuracy: 0.5812 - val_loss: 1.7421 - val_categorical_accuracy: 0.4584
Epoch 4/20
8419/8419 [==============================] - 44s 5ms/step - loss: 1.4332 - categorical_accuracy: 0.6212 - val_loss: 1.5899 - val_categorical_accuracy: 0.5625
Epoch 5/20
8419/8419 [==============================] - 44s 5ms/step - loss: 1.2753 - categorical_accuracy: 0.6572 - val_loss: 1.6155 - val_categorical_accuracy: 0.5544
Epoch 6/20
8419/8419 [==============================] - 44s 5ms/step - loss: 1.0977 - categorical_accuracy:

In [9]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=5e-3),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 39994, 32)         22432     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7998, 32)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 7992, 32)          7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 91)                3003      
Total params: 34,462,335
Trainable params: 34,462,335
Non-trainable params: 0
________________________________________________________________

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 39994, 64)         44864     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7998, 64)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 7992, 32)          14368     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 91)                3003      
Total params: 34,491,935
Trainable params: 34,491,935
Non-trainable params: 0
________________________________________________________________

In [11]:
from keras.layers import Flatten
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0309 20:38:09.389264 139653295728448 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 39994, 32)         22432     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 7998, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 255936)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65519872  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 91)                23387     
Total para